In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [10]:
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available



In [11]:

if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=phones&crid=G2YIV54V0VQ&sprefix=phone%2Caps%2C423&ref=nb_sb_noss_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [12]:
amazon_df

,title,price,rating,reviews,availability
0,VTECH VG101-21 DECT 6.0 2-Handset Cordless Pho...,,4.3 out of 5 stars,"4,500 ratings",In Stock
1,AT&T BL102-3 DECT 6.0 3-Handset Cordless Phone...,,4.3 out of 5 stars,"22,983 ratings",In Stock
2,AT&T DL72219 DECT 6.0 2-Handset Cordless Phone...,,4.2 out of 5 stars,"5,704 ratings",In Stock
3,"Apple iPhone XR, US Version, 64GB, Blue - Unlo...",,4.5 out of 5 stars,"59,844 ratings",Only 3 left in stock - order soon.
4,"Apple iPhone 8, 64GB, Gold - Unlocked (Renewed)",,4.3 out of 5 stars,"52,176 ratings",In Stock.
5,"Apple iPhone 11, US Version, 64GB, Yellow - Un...",,4.3 out of 5 stars,"40,099 ratings",Only 2 left in stock - order soon.
6,"Apple iPhone 8 Plus, US Version, 64GB, Silver ...",,4.4 out of 5 stars,"26,194 ratings",Only 7 left in stock - order soon.
7,"Apple iPhone X, US Version, 64GB, Silver - Unl...",,4.2 out of 5 stars,"23,391 ratings",In Stock.
8,"Apple iPhone XS, 64GB, Silver - Unlocked (Rene...",,4.3 out of 5 stars,"19,929 ratings",In Stock.
9,"Samsung Galaxy S10+, 128GB, Prism Black - AT&T...",,4.4 out of 5 stars,"16,849 ratings",Only 2 left in stock - order soon.
